In [0]:
import cv2
import os
import imageio
import numpy as np
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, adam
from keras.regularizers import l2
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
from keras.callbacks import Callback, LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import print_summary, to_categorical
from keras import backend as K
from  skimage import transform
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

BATCH_SIZE = 100
NUM_CLASSES = 2
EPOCHS = 10

In [0]:
def load_data():
    labels = ['apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl',
              'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle', 'chair',
              'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 'dolphin',
              'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp',
              'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain',
              'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 'pickup_truck',
              'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 'possum', 'rabbit', 'raccoon', 'ray', 'road',
              'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 'spider',
              'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 'tank', 'telephone', 'television', 'tiger',
              'tractor', 'train', 'trout', 'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman',
              'worm']

    # Get a list of the indexes we need
    people_indexes = [labels.index("boy"), labels.index("girl"), labels.index("man"), labels.index("woman")]
    cars = labels.index("streetcar")

    (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
    
    # Load CIFAR training data
    i = 0
    X_train_list = []
    Y_train_list = []
    
    for img in x_train:
        if y_train[i, 0] in people_indexes:
            X_train_list.append(img)
            Y_train_list.append(np.array([1, 0]))

        if y_train[i, 0] == cars:
            X_train_list.append(img)
            Y_train_list.append(np.array([0, 1]))

        i += 1

    # Load CIFAR validation data
    i = 0
    X_test_list = []
    Y_test_list = []
    
    for img in x_test:
        if y_train[i, 0] in people_indexes:
            X_test_list.append(img)
            Y_test_list.append(np.array([1, 0]))

        if y_train[i, 0] == cars:
            X_test_list.append(img)
            Y_test_list.append(np.array([0, 1]))


        i += 1     
    
    X_train_list = np.asarray(X_train_list)
    Y_train_list = np.asarray(Y_train_list)
    X_test_list = np.asarray(X_test_list)
    Y_test_list = np.asarray(Y_test_list)
    
    
    

    return (X_train_list, Y_train_list), (X_test_list, Y_test_list)

In [0]:

# Load training and validation data
(x_train, y_train), (x_test, y_test) = load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

Resizing input, Resnet50 is trained on images with the size of 244,244,3 while CIFAR's data is much smaler(32, 32, 3)

In [0]:
print("Resizing input data")
new_shape = (224, 224, 3)
x_train = np.asarray([transform.resize(image, new_shape) for image in x_train])
print(x_train.shape)

x_test = np.asarray([transform.resize(image, new_shape) for image in x_test])
print(x_test.shape)

Loading ResNet50 while excluding the last layer  
Adding a new Softmax layer on top

In [0]:
model = Sequential()

model.add(resnet50.ResNet50(include_top=False, weights='imagenet', pooling='avg'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model.add(Dense(NUM_CLASSES, activation='softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

model.summary()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'model.{epoch:03d}.h5'

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

callbacks = [checkpoint]

In [0]:
model.fit(x_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=(x_test, y_test),
          shuffle=True, callbacks=callbacks)